In [1]:
import requests
import os

# The direct link you provided
url = 'https://usf.box.com/shared/static/ohjs9p90r0fgt3u3pwt10o6m249i0j12'
output_filename = 'dbNSFP4.9a.zip'

print(f"🚀 Starting download of {output_filename}...")
print("This file is large (10GB+), so please wait. Do not close the tab.")

try:
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_filename, 'wb') as f:
            downloaded = 0
            chunk_size = 1024 * 1024 * 10  # 10 MB chunks
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    # Print status every 1GB (approx)
                    if downloaded % (1024 * 1024 * 1024) < chunk_size:
                        print(f"Downloaded: {downloaded / (1024*1024*1024):.2f} GB...")
                        
    print(f"\n✅ Download Complete! Saved as: {output_filename}")
    print(f"File size: {os.path.getsize(output_filename) / (1024*1024*1024):.2f} GB")

except Exception as e:
    print(f"\n❌ Error downloading: {e}")

🚀 Starting download of dbNSFP4.9a.zip...
This file is large (10GB+), so please wait. Do not close the tab.
Downloaded: 1.01 GB...
Downloaded: 2.00 GB...
Downloaded: 3.01 GB...
Downloaded: 4.00 GB...
Downloaded: 5.00 GB...
Downloaded: 6.01 GB...
Downloaded: 7.00 GB...
Downloaded: 8.01 GB...
Downloaded: 9.00 GB...
Downloaded: 10.00 GB...
Downloaded: 11.01 GB...
Downloaded: 12.00 GB...
Downloaded: 13.01 GB...
Downloaded: 14.00 GB...
Downloaded: 15.00 GB...
Downloaded: 16.01 GB...
Downloaded: 17.00 GB...
Downloaded: 18.01 GB...
Downloaded: 19.00 GB...
Downloaded: 20.00 GB...
Downloaded: 21.01 GB...
Downloaded: 22.00 GB...
Downloaded: 23.01 GB...
Downloaded: 24.00 GB...
Downloaded: 25.00 GB...
Downloaded: 26.01 GB...
Downloaded: 27.00 GB...
Downloaded: 28.01 GB...
Downloaded: 29.00 GB...
Downloaded: 30.00 GB...
Downloaded: 31.01 GB...
Downloaded: 32.00 GB...
Downloaded: 33.01 GB...
Downloaded: 34.00 GB...
Downloaded: 35.00 GB...

✅ Download Complete! Saved as: dbNSFP4.9a.zip
File size: 35.9

In [2]:
!unzip dbNSFP4.9a.zip -d dbnsfp_data

Archive:  dbNSFP4.9a.zip
 extracting: dbnsfp_data/dbNSFP4.8a.readme.txt  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr1.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr10.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr11.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr12.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr13.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr14.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr15.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr16.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr17.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr18.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr19.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr2.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr20.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr21.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr22.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr3.gz  
 extracting: dbnsfp_data/dbNSFP4.8a_variant.chr4.

In [2]:
!pip install pyarrow

In [1]:
import pandas as pd
import glob
import os
import re

# ================= CONFIGURATION =================
# 1. Your LLM Test Set (The one with "text" prompt)
TEST_SET_PATH = "test_unseen_variant_level.parquet"

# 2. Your RAW ClinVar Data (The key to find coordinates)
# Usually named 'variant_summary.txt'
RAW_CLINVAR_PATH = "data/variant_summary.txt" 

# 3. dbNSFP Folder (Where you unzipped chr1, chr2, etc.)
DBNSFP_FOLDER = "dbnsfp_data"

# 4. Output Filename
OUTPUT_FILE = "pathopreter_vs_sota.csv"
# =================================================

def extract_variant_nm(text):
    """
    Extracts the NM_ identifier from the LLM prompt.
    Example Input: "Variant: NM_201384.3(PLEC):c.2406C>A..."
    Output: "NM_201384.3(PLEC):c.2406C>A"
    """
    try:
        # Regex to capture everything after "Variant: " until the next newline or space
        match = re.search(r"Variant:\s+([^\n\s]+)", text)
        if match:
            return match.group(1).strip()
    except:
        pass
    return None

print("🚀 STARTING BENCHMARK EXTRACTION")

# --- STEP 1: Load Test Set & Extract Names ---
print(f"\n[1/4] Loading Test Set from {TEST_SET_PATH}...")
df_test = pd.read_parquet(TEST_SET_PATH)

# Extract the Name (NM_...) from the prompt text
df_test['Name'] = df_test['text'].apply(extract_variant_nm)

print(f"Loaded {len(df_test)} rows.")
print(f"Sample Extracted Name: {df_test['Name'].iloc[0]}")

# --- STEP 2: Load ClinVar Summary to get Coordinates ---
print(f"\n[2/4] Loading ClinVar Summary (Mapping Name -> Chrom/Pos)...")
print("This can take 1-2 minutes...")

# We only need specific columns to map the name to coordinates
# variant_summary.txt is tab-separated
df_raw = pd.read_csv(
    RAW_CLINVAR_PATH, 
    sep='\t',
    usecols=['Name', 'Chromosome', 'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF'],
    dtype={'Chromosome': str, 'PositionVCF': str}, # Keep as strings to avoid type errors
    low_memory=False
)

# Filter ClinVar to only include the variants in our test set (Optimization)
# This drastically reduces the dataframe size for the merge
test_names = set(df_test['Name'].unique())
df_raw = df_raw[df_raw['Name'].isin(test_names)]

print(f"Found {len(df_raw)} matching records in variant_summary.txt")

# Merge coordinates into the test set
df_mapped = df_test.merge(df_raw, on='Name', how='left')

# Drop rows that failed to map (if any)
df_ready = df_mapped.dropna(subset=['Chromosome', 'PositionVCF'])

# Create a "Lookup Key" for dbNSFP: "1:12345:A:G"
# Note: dbNSFP usually doesn't use "chr" prefix in the data columns, so we strip it if present
df_ready['Chromosome'] = df_ready['Chromosome'].str.replace("chr", "", case=False)

df_ready['lookup_key'] = (
    df_ready['Chromosome'] + ":" + 
    df_ready['PositionVCF'] + ":" + 
    df_ready['ReferenceAlleleVCF'] + ":" + 
    df_ready['AlternateAlleleVCF']
)

# Create a set for O(1) instant lookup
lookup_set = set(df_ready['lookup_key'])
print(f"Successfully mapped {len(df_ready)} variants to genomic coordinates.")


# --- STEP 3: Scan dbNSFP for Scores ---
print(f"\n[3/4] Scanning dbNSFP database for CADD & REVEL scores...")

found_scores = []
dbnsfp_files = glob.glob(os.path.join(DBNSFP_FOLDER, "*variant.chr*"))

# Columns we want from dbNSFP
# Note: Column names might vary slightly by version. 
# Standard 4.x names: '#chr', 'pos(1-based)', 'ref', 'alt', 'CADD_phred', 'REVEL_score'
cols_to_use = ['#chr', 'pos(1-based)', 'ref', 'alt', 'CADD_phred', 'REVEL_score']

for f_path in sorted(dbnsfp_files):
    chr_name = os.path.basename(f_path).split('.')[-1] # e.g., chr1
    print(f"  > Scanning {chr_name}...", end="\r")
    
    try:
        # Read in chunks (100k rows) to save RAM
        chunk_iter = pd.read_csv(
            f_path, 
            sep='\t', 
            usecols=lambda c: c in cols_to_use, # robust column loading
            chunksize=100000, 
            low_memory=False,
            dtype={'#chr': str, 'pos(1-based)': str}
        )
        
        for chunk in chunk_iter:
            # Create key
            chunk['lookup_key'] = (
                chunk['#chr'] + ":" + 
                chunk['pos(1-based)'] + ":" + 
                chunk['ref'] + ":" + 
                chunk['alt']
            )
            
            # Filter: Keep only rows that match our test set
            matches = chunk[chunk['lookup_key'].isin(lookup_set)]
            
            if not matches.empty:
                # Keep only key and scores
                found_scores.append(matches[['lookup_key', 'CADD_phred', 'REVEL_score']])
                
    except Exception as e:
        print(f"\n    Skipping {chr_name}: {e}")

print(f"\n  > Finished scanning.")

# --- STEP 4: Merge & Save ---
print(f"\n[4/4] Merging and Saving...")

if found_scores:
    df_scores = pd.concat(found_scores)
    
    # Merge scores back into your main dataframe
    final_df = df_ready.merge(df_scores, on='lookup_key', how='left')
    
    # Save
    final_df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n✅ SUCCESS! Results saved to: {OUTPUT_FILE}")
    print(f"Matched scores for {len(df_scores)} variants.")
    print("Check the 'CADD_phred' and 'REVEL_score' columns in the output file.")
else:
    print("\n❌ CRITICAL: No matching scores found.")
    print("Check if dbNSFP files are unzipped correctly or if coordinate format matches.")

🚀 STARTING BENCHMARK EXTRACTION

[1/4] Loading Test Set from test_unseen_variant_level.parquet...
Loaded 55376 rows.
Sample Extracted Name: ABCC9,

[2/4] Loading ClinVar Summary (Mapping Name -> Chrom/Pos)...
This can take 1-2 minutes...
Found 38298 matching records in variant_summary.txt
Successfully mapped 38298 variants to genomic coordinates.

[3/4] Scanning dbNSFP database for CADD & REVEL scores...


/tmp/ipykernel_41208/3461606888.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready['Chromosome'] = df_ready['Chromosome'].str.replace("chr", "", case=False)
/tmp/ipykernel_41208/3461606888.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready['lookup_key'] = (


  > Scanning gz...
  > Finished scanning.

[4/4] Merging and Saving...

✅ SUCCESS! Results saved to: pathopreter_vs_sota.csv
Matched scores for 1927 variants.
Check the 'CADD_phred' and 'REVEL_score' columns in the output file.


In [5]:
import pandas as pd
import numpy as np

# Load your raw results
df = pd.read_csv("pathopreter_vs_sota.csv", low_memory=False)

# 1. Cleaning Function (Improved)
def get_max_score(value):
    """
    Parses '0.5;0.6' or '.;0.5' strings.
    Returns None if no valid number exists.
    """
    try:
        parts = str(value).split(';')
        valid_nums = []
        for x in parts:
            x = x.strip()
            # Check for real numbers, ignore dots/empty
            if x != '.' and x != '' and x.lower() != 'nan':
                try:
                    valid_nums.append(float(x))
                except:
                    pass
        
        if not valid_nums:
            return None # Return None instead of 0.0 to distinguish "Missing" from "Safe"
            
        return max(valid_nums)
    except:
        return None

# Apply cleaning
df['CADD_Final'] = df['CADD_phred'].apply(get_max_score)
df['REVEL_Final'] = df['REVEL_score'].apply(get_max_score)

# 2. CREATE TWO SEPARATE DATASETS (Fair Comparison)
# REVEL is picky, CADD is broad. We can't use the same N for both if we want to be fair.

# --- DATASET A: The "REVEL-Applicable" Subset (Missense Only) ---
df_revel = df.dropna(subset=['REVEL_Final'])

# --- DATASET B: The "CADD-Applicable" Subset (Almost everything) ---
df_cadd = df.dropna(subset=['CADD_Final'])

# 3. Calculate Metrics
def calculate_metrics(dataframe, score_col, threshold):
    # Predict
    dataframe['Pred'] = dataframe[score_col].apply(lambda x: "Pathogenic" if x >= threshold else "Benign")
    
    # Overall Accuracy
    acc = (dataframe['Pred'] == dataframe['clean_label']).mean()
    
    # Recall (Sensitivity) - Only on Pathogenic rows
    path_rows = dataframe[dataframe['clean_label'] == 'Pathogenic']
    if len(path_rows) > 0:
        recall = (path_rows['Pred'] == 'Pathogenic').mean()
    else:
        recall = 0.0
        
    return acc, recall, len(dataframe), len(path_rows)

# Get Stats
revel_acc, revel_recall, revel_n, revel_path_n = calculate_metrics(df_revel, 'REVEL_Final', 0.5)
cadd_acc, cadd_recall, cadd_n, cadd_path_n = calculate_metrics(df_cadd, 'CADD_Final', 20.0)

print("="*40)
print("      FINAL CORRECTED BENCHMARK")
print("="*40)

print(f"\n--- REVEL (Missense Variants Only) ---")
print(f"Sample Size (N): {revel_n} (Pathogenic count: {revel_path_n})")
print(f"REVEL Recall:    {revel_recall*100:.1f}%")
print(f"REVEL Accuracy:  {revel_acc*100:.1f}%")

print(f"\n--- CADD (All Variants) ---")
print(f"Sample Size (N): {cadd_n} (Pathogenic count: {cadd_path_n})")
print(f"CADD Recall:     {cadd_recall*100:.1f}%")
print(f"CADD Accuracy:   {cadd_acc*100:.1f}%")

print(f"\n--- YOUR PATHOPRETER ---")
print(f"Recall: ~94.0% (On full dataset)")
print("="*40)

      FINAL CORRECTED BENCHMARK

--- REVEL (Missense Variants Only) ---
Sample Size (N): 243 (Pathogenic count: 18)
REVEL Recall:    27.8%
REVEL Accuracy:  84.4%

--- CADD (All Variants) ---
Sample Size (N): 1937 (Pathogenic count: 1592)
CADD Recall:     99.0%
CADD Accuracy:   94.7%

--- YOUR PATHOPRETER ---
Recall: ~94.0% (On full dataset)


/tmp/ipykernel_41208/3374293388.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Pred'] = dataframe[score_col].apply(lambda x: "Pathogenic" if x >= threshold else "Benign")
/tmp/ipykernel_41208/3374293388.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Pred'] = dataframe[score_col].apply(lambda x: "Pathogenic" if x >= threshold else "Benign")


In [6]:
# Sanity Check: See exactly what the cleaner did
print("--- RAW vs CLEANED Check ---")
for i, row in df_revel.head(5).iterrows():
    raw = row['REVEL_score']
    clean = row['REVEL_Final']
    pred = row['Pred']
    print(f"Raw: {str(raw):<20} -> Cleaned: {clean:<5} -> Prediction: {pred}")

--- RAW vs CLEANED Check ---
Raw: .;0.788;0.788        -> Cleaned: 0.788 -> Prediction: Pathogenic
Raw: 0.106;.              -> Cleaned: 0.106 -> Prediction: Benign
Raw: 0.027                -> Cleaned: 0.027 -> Prediction: Benign
Raw: 0.188;0.188          -> Cleaned: 0.188 -> Prediction: Benign
Raw: 0.027;.;0.027;.;.    -> Cleaned: 0.027 -> Prediction: Benign
